In [1]:
import pyspark

In [14]:
# from pyspark.sql.functions import concat 
# from pyspark.sql.functions import when 
# from pyspark.sql.functions import lit 
# from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql import Column
from pyspark.sql.window import Window

In [3]:
from pyspark.sql import SparkSession

In [4]:
session = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
dataframereader = session.read

In [6]:
df = dataframereader.csv("mycsv1.csv",header=True)

In [7]:
df= df.withColumnRenamed("_c0","Skey")

In [108]:
df = df.withColumn("Proba",lit(""))

df.withColumn("Probability",concat("Proba",
when(df.max_consec_weeks>1,"alpha|").otherwise("beta|")   
)).select("Probability").show(2,False)

+-----------+
|Probability|
+-----------+
|beta|      |
|alpha|     |
+-----------+
only showing top 2 rows



In [141]:
#df.filter(col("distinct_periods")==(1)).show()
df1= df.select("CustomerID","weeks_shopped")
df2= df.select("total_qty","med_spend")
#cond = Column(df1.CustomerID==df2.total_qty)
df1.join(df2,df1.CustomerID==df2.total_qty,"inner").show()


+----------+-------------+---------+---------+
|CustomerID|weeks_shopped|total_qty|med_spend|
+----------+-------------+---------+---------+
|     12540|           16|    12540|      0.0|
|     12665|            1|    12665|  3962.85|
|     12897|            2|    12897|   329.52|
|     13476|            2|    13476|  1680.88|
|     16232|            3|    16232|    793.5|
|     16352|            2|    16352|  4050.61|
+----------+-------------+---------+---------+



In [143]:
dataframe_mysql = session.read\
    .format("jdbc")\
    .option("url", "jdbc:mysql://localhost/arms")\
    .option("driver", "com.mysql.jdbc.Driver")\
    .option("dbtable", "users").option("user", "root")\
    .option("password", "Programming1!").load()



In [144]:
dataframe_mysql.show()

+---------+--------------------+----------+---------+--------------------+------------+-------------------+--------------+
| username|            password|first_name|Last_name|               email|phone_number|         last_login|       api_key|
+---------+--------------------+----------+---------+--------------------+------------+-------------------+--------------+
|amansingh|3192b26ef8d0595ba...|      Aman|    Singh|singhaman11415@gm...|  8770262013|2020-06-06 18:49:41|abhikuchnhihai|
|     root|              dummy1|         v|   prasad|   vprasad@gmail.com|          99|2020-06-07 14:19:17|         nulll|
|  vprasad|               dummy|         v|   prasad|   vprasad@gmail.com|          99|2020-07-05 20:59:30|         nulll|
+---------+--------------------+----------+---------+--------------------+------------+-------------------+--------------+



In [150]:
sqlData = [("first","123","spark","sql","sparkk",999,"2020-06-06 18:49:41","")]
sqldf = session.createDataFrame(sqlData,["username","password","first_name","Last_name","email","phone_number","last_login","api_key"])
sqldf.write.format('jdbc').options(
      url='jdbc:mysql://localhost/arms',
      driver='com.mysql.jdbc.Driver',
      dbtable='users',
      user='root',
      password='Programming1!').mode('append').save()

In [157]:
chcekOperator = dataframe_mysql.withColumn("Verified",when(dataframe_mysql.username!="vprasad",lit("True"))
                                           .otherwise(lit("false")))\
                                .withColumn("Starting",when(dataframe_mysql.username.startswith("vp"),lit("True"))#startswith
                                           .otherwise(lit("false")))
chcekOperator.select("Verified","username","Starting").show()

+--------+---------+--------+
|Verified| username|Starting|
+--------+---------+--------+
|    True|amansingh|   false|
|    True|    first|   false|
|    True|     root|   false|
|   false|  vprasad|    True|
+--------+---------+--------+



In [23]:
df.select("skey","CustomerID","shabit","spend_desc","visit_desc").show(10,False)

+----+----------+------+----------+-------------+
|skey|CustomerID|shabit|spend_desc|visit_desc   |
+----+----------+------+----------+-------------+
|0   |12346     |PO    |1.High    |6.Hardly Ever|
|1   |12347     |PR    |1.High    |1.Daily      |
|2   |12348     |PO    |2.Medium  |5.Now & Then |
|3   |12349     |PO    |1.High    |6.Hardly Ever|
|4   |12350     |UN    |2.Medium  |6.Hardly Ever|
|5   |12352     |VL    |2.Medium  |1.Daily      |
|6   |12353     |UN    |3.Low     |6.Hardly Ever|
|7   |12354     |PO    |1.High    |6.Hardly Ever|
|8   |12355     |UN    |2.Medium  |6.Hardly Ever|
|9   |12356     |UN    |2.Medium  |6.Hardly Ever|
+----+----------+------+----------+-------------+
only showing top 10 rows



In [37]:
#query1-> find customer in each shabit
'''rnkdf = df.withColumn("rankpk",F.dense_rank().over(Window.partitionBy("CustomerID").orderBy("CustomerID"))) \
.select("rankpk","CustomerID").filter("rankpk>1").show(10,False)'''
df.groupBy('shabit').count().withColumnRenamed("count","custCount").filter("custCount>1") \
.show()


+------+---------+
|shabit|custCount|
+------+---------+
|    VL|      445|
|    PO|     1061|
|    UN|     2697|
|    PR|      136|
+------+---------+



In [5]:
dataframereader1 = session.read

In [6]:
dfOutward = dataframereader1.csv("Outward.csv",header=True)
dfOutward.show(10,False)

+-------+----------+---------+
|docType|supplyType|InvoiceID|
+-------+----------+---------+
|INV    |TAX       |9812INV  |
|INV    |TAX       |9812INV  |
|ABC    |TAX       |9812INV  |
|INV    |TAX       |9813INV  |
|INV    |INV       |9813INV  |
|INV    |NON       |9814INV  |
|INV    |TAX       |9814INV  |
|INV    |TAX       |9814INV  |
+-------+----------+---------+



In [28]:
'''dfOutward.groupBy(["InvoiceID"]).count("supplyType").withColumnRenamed("count","mulSupply")\
.show(20,False)'''

w = Window.partitionBy("InvoiceID").orderBy("supplyType")

dupInvoice = dfOutward.withColumn("srank",F.dense_rank().over(w)) \
.filter("srank>1").select("InvoiceID").withColumnRenamed("InvoiceID","InvoiceIDD").dropDuplicates()

on=dfOutward.InvoiceID==dupInvoice.InvoiceIDD

withNull = dfOutward.join(dupInvoice,on,"leftouter")
withNull.withColumn("BR2", F.when(withNull.InvoiceIDD is not None and withNull.supplyType!="INV" ,F.lit("2") ).otherwise("1")).show()


+-------+----------+---------+----------+---+
|docType|supplyType|InvoiceID|InvoiceIDD|BR2|
+-------+----------+---------+----------+---+
|    INV|       TAX|  9812INV|      null|  2|
|    INV|       TAX|  9812INV|      null|  2|
|    ABC|       TAX|  9812INV|      null|  2|
|    INV|       TAX|  9813INV|   9813INV|  2|
|    INV|       INV|  9813INV|   9813INV|  1|
|    INV|       NON|  9814INV|   9814INV|  2|
|    INV|       TAX|  9814INV|   9814INV|  2|
|    INV|       TAX|  9814INV|   9814INV|  2|
+-------+----------+---------+----------+---+



In [66]:
help(dfOutward.dropDuplicates)

Help on method dropDuplicates in module pyspark.sql.dataframe:

dropDuplicates(subset=None) method of pyspark.sql.dataframe.DataFrame instance
    Return a new :class:`DataFrame` with duplicate rows removed,
    optionally only considering certain columns.
    
    For a static batch :class:`DataFrame`, it just drops duplicate rows. For a streaming
    :class:`DataFrame`, it will keep all data across triggers as intermediate state to drop
    duplicates rows. You can use :func:`withWatermark` to limit how late the duplicate data can
    be and system will accordingly limit the state. In addition, too late data older than
    watermark will be dropped to avoid any possibility of duplicates.
    
    :func:`drop_duplicates` is an alias for :func:`dropDuplicates`.
    
    >>> from pyspark.sql import Row
    >>> df = sc.parallelize([ \
    ...     Row(name='Alice', age=5, height=80), \
    ...     Row(name='Alice', age=5, height=80), \
    ...     Row(name='Alice', age=10, height=80)]).to

In [ ]:
!conda install --yes plotly